In [1]:
import pandas
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import *
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.optimizers import adam

Using TensorFlow backend.


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc

import keras as k
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import EarlyStopping, ModelCheckpoint

import cv2
from tqdm import tqdm

from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import KFold
from sklearn.metrics import fbeta_score
import time

/Users/zhenzhenzhong/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
x_train = []
x_test = []
y_train = []

df_train = pd.read_csv('/Users/zhenzhenzhong/Downloads/train_v2.csv')
df_test = pd.read_csv('/Users/zhenzhenzhong/Downloads/sample_submission_v2.csv')

flatten = lambda l: [item for sublist in l for item in sublist]
labels = list(set(flatten([l.split(' ') for l in df_train['tags'].values])))

labels = ['blow_down',
 'bare_ground',
 'conventional_mine',
 'blooming',
 'cultivation',
 'artisinal_mine',
 'haze',
 'primary',
 'slash_burn',
 'habitation',
 'clear',
 'road',
 'selective_logging',
 'partly_cloudy',
 'agriculture',
 'water',
 'cloudy']

In [4]:
label_map = {'agriculture': 14,
 'artisinal_mine': 5,
 'bare_ground': 1,
 'blooming': 3,
 'blow_down': 0,
 'clear': 10,
 'cloudy': 16,
 'conventional_mine': 2,
 'cultivation': 4,
 'habitation': 9,
 'haze': 6,
 'partly_cloudy': 13,
 'primary': 7,
 'road': 11,
 'selective_logging': 12,
 'slash_burn': 8,
 'water': 15}

In [5]:
for f, tags in tqdm(df_train.values, miniters=1000):
    img = cv2.imread('/Users/zhenzhenzhong/Downloads/train-jpg/{}.jpg'.format(f))
    targets = np.zeros(17)
    for t in tags.split(' '):
        targets[label_map[t]] = 1 
    x_train.append(cv2.resize(img, (32, 32)))
    y_train.append(targets)
    img = cv2.flip(img, 0)  # flip vertically
    x_train.append(cv2.resize(img, (32, 32)))
    y_train.append(targets)
    img = cv2.flip(img, 1)  # flip horizontally
    x_train.append(cv2.resize(img, (32, 32)))
    y_train.append(targets)
    img = cv2.flip(img, 0)  # flip vertically
    x_train.append(cv2.resize(img, (32, 32)))
    y_train.append(targets)

for f, tags in tqdm(df_test.values, miniters=1000):
    img = cv2.imread('/Users/zhenzhenzhong/Downloads/test-jpg/{}.jpg'.format(f))
    x_test.append(cv2.resize(img, (32, 32)))   

    
y_train = np.array(y_train, np.uint8)
x_train = np.array(x_train, np.float32) / 255.
x_test  = np.array(x_test, np.float32) / 255.

print(x_train.shape)
print(y_train.shape)

100%|██████████| 61191/61191 [01:57<00:00, 522.20it/s]


(161916, 32, 32, 3)
(161916, 17)


In [20]:
len(y_train)

161916

In [6]:
trainx, validx, trainy, validy = train_test_split(x_train, y_train, test_size=0.25, random_state=1)

In [10]:
#from sklearn.model_selection import StratifiedShuffleSplit
#trn = []
#val = []
#index = np.arange(len(y_train))

#for i in tqdm(range(0,17)):
#    sss = StratifiedShuffleSplit(n_splits=2, test_size=0.2, random_state=i)
#    for train_index, valid_index in sss.split(index,y_train[:,i]):
#        trainx, validx = index[train_index], index[valid_index]
    # to ensure there is no repetetion within each split and between the splits
#    trn = trn + list(set(list(trainx)) - set(trn) - set(val))
#    val = val + list(set(list(validx)) - set(val) - set(trn))

100%|██████████| 17/17 [00:02<00:00,  7.80it/s]


In [20]:
#trainx=x_train[trn]

In [17]:
#trainy=y_train[trn]

In [22]:
#validx=x_train[val]

In [23]:
#validy=y_train[val]

In [8]:
# https://www.kaggle.com/c/planet-understanding-the-amazon-from-space/discussion/32475
import numpy as np
from sklearn.metrics import fbeta_score

def optimise_f2_thresholds(y, p, verbose=True, resolution=100):
    def mf(x):
        p2 = np.zeros_like(p)
        for i in range(17):
            p2[:, i] = (p[:, i] > x[i]).astype(np.int)
        score = fbeta_score(y, p2, beta=2, average='samples')
        return score

    x = [0.2]*17
    for i in range(17):
        best_i2 = 0
        best_score = 0
        for i2 in range(resolution):
            i2 /= resolution
            x[i] = i2
            score = mf(x)
            if score > best_score:
                best_i2 = i2
                best_score = score
        x[i] = best_i2
        if verbose:
            print(i, best_i2, best_score)

    return x

In [9]:
traindatagenerator = ImageDataGenerator(width_shift_range=0.1,height_shift_range=0.1,rotation_range=15,zoom_range=0.1)

In [10]:
validdatagenerator = ImageDataGenerator(width_shift_range=0.1,height_shift_range=0.1,rotation_range=15,zoom_range=0.1)

In [11]:
batchsize=32
train_generator=traindatagenerator.flow(trainx,trainy, batch_size=batchsize) 
valid_generator=validdatagenerator.flow(validx,validy, batch_size=batchsize) 

In [18]:
model = Sequential()
model.add(BatchNormalization(input_shape=(32, 32, 3)))
model.add(Conv2D(32, kernel_size=(2, 2), padding='same', activation='relu'))
model.add(Conv2D(32, kernel_size=(2, 2), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, kernel_size=(2, 2), padding='same', activation='relu'))
model.add(Conv2D(64, kernel_size=(2, 2), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, kernel_size=(2, 2), padding='same', activation='relu'))
model.add(Conv2D(128, kernel_size=(2, 2), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(256, kernel_size=(2, 2), padding='same', activation='relu'))
model.add(Conv2D(256, kernel_size=(2, 2), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(17, activation='sigmoid'))


In [19]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [21]:
from keras.layers.normalization import BatchNormalization

nfolds = 3

num_fold = 0
sum_score = 0

yfull_test = []
yfull_train =[]

kf = KFold(len(y_train), n_folds=nfolds, shuffle=True, random_state=1)

for train_index, test_index in kf:
        start_time_model_fitting = time.time()
        
        X_train = x_train[train_index]
        Y_train = y_train[train_index]
        X_valid = x_train[test_index]
        Y_valid = y_train[test_index]

        num_fold += 1
        print('Start KFold number {} from {}'.format(num_fold, nfolds))
        print('Split train: ', len(X_train), len(Y_train))
        print('Split valid: ', len(X_valid), len(Y_valid))
        
        kfold_weights_path = os.path.join('', 'weights_kfold_0709aug32_' + str(num_fold) + '.h5')

        callbacks = [
            EarlyStopping(monitor='val_loss', patience=2, verbose=0),
            ModelCheckpoint(kfold_weights_path, monitor='val_loss', save_best_only=True, verbose=0)]
        
        train_generator=traindatagenerator.flow(X_train,Y_train, batch_size=batchsize) 
        valid_generator=validdatagenerator.flow(X_valid,Y_valid, batch_size=batchsize) 
        
        model.fit_generator(train_generator, steps_per_epoch=int(len(trainy)/batchsize), epochs=10, 
                    validation_data=valid_generator, validation_steps=int(len(validy)/batchsize), callbacks=callbacks)
        
        #model.fit(x = X_train, y= Y_train, validation_data=(X_valid, Y_valid),
        #          batch_size=128,verbose=2, nb_epoch=10,callbacks=callbacks,
        #          shuffle=True)
        
        if os.path.isfile(kfold_weights_path):
            model.load_weights(kfold_weights_path)
        
        p_valid = model.predict(X_valid, batch_size = 128, verbose=2)
        print(fbeta_score(Y_valid, np.array(p_valid) > 0.2, beta=2, average='samples'))
        print("Optimizing prediction threshold")
        print(optimise_f2_thresholds(Y_valid, p_valid))
        
        #p_test = model.predict(x_train, batch_size = 128, verbose=2)
        #yfull_train.append(p_test)
        
        p_test = model.predict(x_test, batch_size = 128, verbose=2)
        yfull_test.append(p_test)

Start KFold number 1 from 3
Split train:  107944 107944
Split valid:  53972 53972
Epoch 1/10
3794/3794 [==============================] - 450s - loss: 0.1712 - acc: 0.9330 - val_loss: 0.1450 - val_acc: 0.9419
Epoch 2/10
3794/3794 [==============================] - 437s - loss: 0.1445 - acc: 0.9436 - val_loss: 0.1327 - val_acc: 0.9474
Epoch 3/10
3794/3794 [==============================] - 439s - loss: 0.1391 - acc: 0.9459 - val_loss: 0.1282 - val_acc: 0.9491
Epoch 4/10
3794/3794 [==============================] - 438s - loss: 0.1358 - acc: 0.9476 - val_loss: 0.1278 - val_acc: 0.9508
Epoch 5/10
3794/3794 [==============================] - 436s - loss: 0.1337 - acc: 0.9488 - val_loss: 0.1256 - val_acc: 0.9512
Epoch 6/10
3794/3794 [==============================] - 431s - loss: 0.1332 - acc: 0.9488 - val_loss: 0.1224 - val_acc: 0.9527
Epoch 7/10
3794/3794 [==============================] - 430s - loss: 0.1322 - acc: 0.9492 - val_loss: 0.1276 - val_acc: 0.9509
Epoch 8/10
3794/3794 [=======

/Users/zhenzhenzhong/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


16 0.22 0.899741423063
[0.07, 0.16, 0.23, 0.09, 0.21, 0.21, 0.24, 0.12, 0.12, 0.22, 0.21, 0.27, 0.29, 0.15, 0.26, 0.2, 0.22]
Start KFold number 2 from 3
Split train:  107944 107944
Split valid:  53972 53972
Epoch 1/10
3794/3794 [==============================] - 435s - loss: 0.1318 - acc: 0.9498 - val_loss: 0.1208 - val_acc: 0.9531
Epoch 2/10
3794/3794 [==============================] - 431s - loss: 0.1314 - acc: 0.9499 - val_loss: 0.1204 - val_acc: 0.9536
Epoch 3/10
3794/3794 [==============================] - 431s - loss: 0.1307 - acc: 0.9501 - val_loss: 0.1214 - val_acc: 0.9537
Epoch 4/10
3794/3794 [==============================] - 432s - loss: 0.1308 - acc: 0.9503 - val_loss: 0.1221 - val_acc: 0.9523
Epoch 5/10
3794/3794 [==============================] - 431s - loss: 0.1309 - acc: 0.9500 - val_loss: 0.1192 - val_acc: 0.9540
Epoch 6/10
3794/3794 [==============================] - 431s - loss: 0.1309 - acc: 0.9499 - val_loss: 0.1242 - val_acc: 0.9520
Epoch 7/10
3794/3794 [=========

In [22]:
result = np.array(yfull_test[0])
for i in range(1, nfolds):
    result += np.array(yfull_test[i])
result /= nfolds
result = pd.DataFrame(result, columns = labels)
result

,blow_down,bare_ground,conventional_mine,blooming,cultivation,artisinal_mine,haze,primary,slash_burn,habitation,clear,road,selective_logging,partly_cloudy,agriculture,water,cloudy
0,1.872437e-03,0.000498,7.460765e-09,1.373595e-02,0.010330,1.391864e-08,0.001173,0.999949,3.239275e-05,0.001717,0.990676,0.010967,5.787405e-03,0.008227,0.020465,0.032263,1.050029e-06
1,3.678523e-03,0.000726,4.098393e-08,1.729150e-02,0.018279,7.192665e-08,0.003133,0.999903,1.422887e-04,0.001872,0.903286,0.012671,6.246494e-03,0.103435,0.033101,0.039177,8.536373e-06
2,3.562820e-04,0.000742,1.766418e-06,9.051621e-05,0.021351,2.128060e-06,0.000777,0.984986,1.638118e-04,0.003356,0.001821,0.030355,1.769761e-04,0.984067,0.070427,0.091791,1.304651e-02
3,1.167239e-02,0.011733,9.534203e-05,1.326426e-02,0.315810,2.433195e-04,0.019065,0.999739,1.005600e-02,0.028786,0.794116,0.081415,1.301061e-02,0.183554,0.406174,0.147583,2.593736e-05
4,3.975067e-08,0.000296,3.648645e-09,1.620509e-07,0.009706,9.882714e-07,0.008204,0.650798,5.142938e-07,0.003351,0.005983,0.018776,2.855325e-07,0.607945,0.048079,0.063424,3.535528e-01
5,1.344805e-03,0.000278,1.646788e-09,1.011689e-02,0.007336,7.568741e-09,0.000908,0.999967,1.547054e-05,0.001153,0.992366,0.007904,4.018424e-03,0.007133,0.014343,0.025972,7.403610e-07
6,6.364558e-03,0.027602,1.133639e-03,3.125021e-03,0.352715,1.079584e-03,0.120022,0.997919,2.472205e-02,0.097743,0.547126,0.216763,7.043591e-03,0.293739,0.754964,0.171866,1.506258e-04
7,3.373712e-14,0.058636,7.743834e-04,1.533854e-10,0.010956,3.608495e-03,0.013059,0.645460,5.923868e-08,0.979945,0.980248,0.985146,1.326537e-05,0.006573,0.263538,0.113878,7.264566e-08
8,1.609536e-03,0.000410,3.103873e-09,1.215388e-02,0.009434,1.090285e-08,0.001264,0.999948,2.118650e-05,0.001508,0.990703,0.009578,5.065561e-03,0.007656,0.018936,0.029345,1.034911e-06
9,3.161958e-05,0.005058,2.418020e-05,6.137769e-05,0.222781,6.398850e-06,0.775685,0.995494,1.044707e-04,0.027296,0.227562,0.133871,3.672401e-04,0.025857,0.492309,0.180389,1.387359e-03


In [23]:
list1=[0.07, 0.16, 0.23, 0.09, 0.21, 0.21, 0.24, 0.12, 0.12, 0.22, 0.21, 0.27, 0.29, 0.15, 0.26, 0.20, 0.22]
list2=[0.11, 0.13, 0.14, 0.11, 0.24, 0.26, 0.22, 0.16, 0.10, 0.23, 0.16, 0.21, 0.11, 0.15, 0.26, 0.21, 0.11]
list3=[0.08, 0.18, 0.14, 0.09, 0.20, 0.13, 0.17, 0.16, 0.10, 0.18, 0.18, 0.18, 0.35, 0.15, 0.28, 0.15, 0.23]

list4=[]
for i in range (0,17):
    list4.append((list1[i]+list2[i]+list3[i])/3)
list4

[0.08666666666666667,
 0.15666666666666668,
 0.17,
 0.09666666666666668,
 0.21666666666666665,
 0.19999999999999998,
 0.21,
 0.1466666666666667,
 0.10666666666666667,
 0.21,
 0.18333333333333335,
 0.21999999999999997,
 0.25,
 0.15,
 0.26666666666666666,
 0.18666666666666668,
 0.18666666666666668]

In [24]:
from tqdm import tqdm
thres = [0.09, 0.16, 0.17, 0.10, 0.22, 0.20, 0.21, 0.15, 0.11, 0.21, 0.18, 0.22, 0.25, 0.15, 0.27, 0.19, 0.19]

preds = []
for i in tqdm(range(result.shape[0]), miniters=1000):
    a = result.ix[[i]]
    a = a.apply(lambda x: x > thres, axis=1)
    a = a.transpose()
    a = a.loc[a[i] == True]
    ' '.join(list(a.index))
    preds.append(' '.join(list(a.index)))

  0%|          | 0/61191 [00:00<?, ?it/s]/Users/zhenzhenzhong/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel_launcher.py:6: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  
100%|██████████| 61191/61191 [02:04<00:00, 492.17it/s]


In [25]:
df_test['tags'] = preds
df_test.head()

,image_name,tags
0,test_0,primary clear
1,test_1,primary clear
2,test_2,primary partly_cloudy
3,test_3,cultivation primary clear partly_cloudy agricu...
4,test_4,primary partly_cloudy cloudy


In [26]:
df_test.to_csv('/Users/zhenzhenzhong/Downloads/submission_keras_b_flip3aug32cv3_0709.csv', index=False)

In [27]:
result = pd.DataFrame(p_test, columns = labels)
result.head()

,blow_down,bare_ground,conventional_mine,blooming,cultivation,artisinal_mine,haze,primary,slash_burn,habitation,clear,road,selective_logging,partly_cloudy,agriculture,water,cloudy
0,2.588243e-03,0.000320,1.091862e-09,1.481173e-02,0.008793,4.473228e-10,0.001240,0.999957,1.959180e-05,0.001397,0.989888,0.011109,8.415202e-03,0.009725,0.016076,0.021897,1.117805e-06
1,2.954638e-03,0.000311,1.386609e-09,1.839050e-02,0.009236,6.897393e-10,0.001281,0.999961,2.469835e-05,0.001446,0.989898,0.012117,1.008532e-02,0.008867,0.015554,0.023183,9.196787e-07
2,8.428615e-04,0.001181,1.761465e-06,1.412812e-04,0.026398,6.245172e-07,0.000045,0.998229,2.803465e-04,0.002840,0.000225,0.035587,3.251595e-04,0.998178,0.076591,0.095932,1.035519e-03
3,1.562071e-02,0.013752,1.061469e-04,1.574097e-02,0.493329,4.168460e-04,0.011469,0.999856,1.710952e-02,0.038141,0.841903,0.091166,2.110185e-02,0.136150,0.460055,0.144409,8.143971e-06
4,1.974799e-13,0.000051,9.944338e-14,5.627120e-12,0.004147,1.224065e-10,0.003840,0.617521,7.522114e-11,0.001687,0.000221,0.013257,5.480670e-11,0.571850,0.027636,0.032162,3.942854e-01


In [28]:
result['image_name'] = df_test['image_name']
result

,blow_down,bare_ground,conventional_mine,blooming,cultivation,artisinal_mine,haze,primary,slash_burn,habitation,clear,road,selective_logging,partly_cloudy,agriculture,water,cloudy,image_name
0,2.588243e-03,0.000320,1.091862e-09,1.481173e-02,0.008793,4.473228e-10,0.001240,0.999957,1.959180e-05,0.001397,0.989888,0.011109,8.415202e-03,0.009725,0.016076,0.021897,1.117805e-06,test_0
1,2.954638e-03,0.000311,1.386609e-09,1.839050e-02,0.009236,6.897393e-10,0.001281,0.999961,2.469835e-05,0.001446,0.989898,0.012117,1.008532e-02,0.008867,0.015554,0.023183,9.196787e-07,test_1
2,8.428615e-04,0.001181,1.761465e-06,1.412812e-04,0.026398,6.245172e-07,0.000045,0.998229,2.803465e-04,0.002840,0.000225,0.035587,3.251595e-04,0.998178,0.076591,0.095932,1.035519e-03,test_2
3,1.562071e-02,0.013752,1.061469e-04,1.574097e-02,0.493329,4.168460e-04,0.011469,0.999856,1.710952e-02,0.038141,0.841903,0.091166,2.110185e-02,0.136150,0.460055,0.144409,8.143971e-06,test_3
4,1.974799e-13,0.000051,9.944338e-14,5.627120e-12,0.004147,1.224065e-10,0.003840,0.617521,7.522114e-11,0.001687,0.000221,0.013257,5.480670e-11,0.571850,0.027636,0.032162,3.942854e-01,test_4
5,2.625038e-03,0.000294,1.020501e-09,1.640413e-02,0.008907,4.670402e-10,0.001204,0.999961,1.930724e-05,0.001426,0.990437,0.011411,9.080939e-03,0.008748,0.015445,0.022031,9.054464e-07,test_5
6,8.051158e-03,0.032447,1.636675e-04,4.719057e-03,0.473996,4.788698e-04,0.037598,0.999281,3.963364e-02,0.106180,0.566043,0.187284,1.058030e-02,0.356948,0.778406,0.117557,1.121264e-05,test_6
7,1.010818e-13,0.087026,1.002515e-03,4.596977e-10,0.024620,5.766818e-03,0.021657,0.656223,1.358753e-07,0.964540,0.978596,0.981105,3.901745e-05,0.005049,0.406527,0.136586,5.296505e-08,test_7
8,2.501650e-03,0.000318,1.095892e-09,1.495567e-02,0.008920,4.630681e-10,0.001101,0.999963,1.877014e-05,0.001414,0.990532,0.011201,8.633331e-03,0.009385,0.015942,0.021806,8.942885e-07,test_8
9,5.333770e-05,0.008870,2.508518e-05,8.574974e-05,0.409572,2.269950e-06,0.765952,0.996386,2.306958e-04,0.036341,0.245571,0.199347,6.106296e-04,0.023645,0.709753,0.174552,3.769206e-04,test_9


In [29]:
result.to_csv('/Users/zhenzhenzhong/Downloads/amazon_submission/0709_keras_b_flip3aug32cv3_prob.csv', index=False)